In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
def convert_to_icd9(dxStr):
    if dxStr.startswith('E'):
        if len(dxStr) > 4: return dxStr[:4] + '.' + dxStr[4:]
        else: return dxStr
    else:
        if len(dxStr) > 3: return dxStr[:3] + '.' + dxStr[3:]
        else: return dxStr
    
def convert_to_3digit_icd9(dxStr):
    if dxStr.startswith('E'):
        if len(dxStr) > 4: return dxStr[:4]
        else: return dxStr
    else:
        if len(dxStr) > 3: return dxStr[:3]
        else: return dxStr

In [3]:
admission_file = Path('../data/ADMISSIONS.csv')
diagnosis_file = Path('../data/DIAGNOSES_ICD.csv')

In [4]:
admissions_df = pd.read_csv(admission_file)
admissions_df['admittime']= pd.to_datetime(admissions_df['admittime'])
admissions_df['dischtime']= pd.to_datetime(admissions_df['dischtime'])
admissions_df['deathtime']= pd.to_datetime(admissions_df['deathtime'])
admissions_df['edregtime']= pd.to_datetime(admissions_df['edregtime'])
admissions_df['edouttime']= pd.to_datetime(admissions_df['edouttime'])
admissions_df.head()

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1
1,12263,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaT,NaT,HEPATITIS B,1,1
2,12265,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,2125-10-07 15:13:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,NaN,UNKNOWN/NOT SPECIFIED,NaT,NaT,SEPSIS,1,1
3,12269,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,2149-05-26 12:08:00,2149-05-26 19:45:00,HUMERAL FRACTURE,0,1
4,12270,10019,177759,2163-05-14 20:43:00,2163-05-15 12:00:00,2163-05-15 12:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,NaT,NaT,ALCOHOLIC HEPATITIS,1,1


In [5]:
diagnosis_df = pd.read_csv(diagnosis_file)
diagnosis_df['icd9_code_converted'] = diagnosis_df['icd9_code'].apply(convert_to_icd9)
diagnosis_df['icd9_code_converted_3digits'] = diagnosis_df['icd9_code'].apply(convert_to_3digit_icd9)
diagnosis_df.head()

,row_id,subject_id,hadm_id,seq_num,icd9_code,icd9_code_converted,icd9_code_converted_3digits
0,112344,10006,142345,1,99591,995.91,995
1,112345,10006,142345,2,99662,996.62,996
2,112346,10006,142345,3,5672,567.2,567
3,112347,10006,142345,4,40391,403.91,403
4,112348,10006,142345,5,42731,427.31,427


In [6]:
codes_per_admission = diagnosis_df.groupby('hadm_id').agg({
    'icd9_code': lambda x: list(x),
    'icd9_code_converted': lambda x: list(x),
    'icd9_code_converted_3digits': lambda x: list(x),
})
combined_df = pd.merge(admissions_df, codes_per_admission, on=['hadm_id'])
admissions_per_subject = combined_df.groupby('subject_id').agg({
    'hadm_id': lambda x: list(x),
    'admittime': lambda x: list(x),
    'diagnosis': lambda x: list(x),
    'icd9_code': lambda x: list(x),
    'icd9_code_converted': lambda x: list(x),
    'icd9_code_converted_3digits': lambda x: list(x),
})
admissions_per_subject['num_admissions'] = admissions_per_subject['hadm_id'].apply(len)
admissions_per_subject.head()

,hadm_id,admittime,diagnosis,icd9_code,icd9_code_converted,icd9_code_converted_3digits,num_admissions
subject_id,,,,,,,
10006,[142345],[2164-10-23 21:09:00],[SEPSIS],"[[99591, 99662, 5672, 40391, 42731, 4280, 4241...","[[995.91, 996.62, 567.2, 403.91, 427.31, 428.0...","[[995, 996, 567, 403, 427, 428, 424, 424, 287,...",1
10011,[105331],[2126-08-14 22:32:00],[HEPATITIS B],"[[570, 07030, 07054, 30401, 2875, 2760]]","[[570, 070.30, 070.54, 304.01, 287.5, 276.0]]","[[570, 070, 070, 304, 287, 276]]",1
10013,[165520],[2125-10-04 23:36:00],[SEPSIS],"[[0389, 41071, 78551, 486, 42731, 20280, 4240,...","[[038.9, 410.71, 785.51, 486, 427.31, 202.80, ...","[[038, 410, 785, 486, 427, 202, 424, 458, 272]]",1
10017,[199207],[2149-05-26 17:19:00],[HUMERAL FRACTURE],"[[81201, 4928, 8028, 8024, 99812, 41511, 2851,...","[[812.01, 492.8, 802.8, 802.4, 998.12, 415.11,...","[[812, 492, 802, 802, 998, 415, 285, 486, E885...",1
10019,[177759],[2163-05-14 20:43:00],[ALCOHOLIC HEPATITIS],"[[0389, 51881, 5770, 30390, 5781, 5845, 2848, ...","[[038.9, 518.81, 577.0, 303.90, 578.1, 584.5, ...","[[038, 518, 577, 303, 578, 584, 284, 572, 785,...",1


In [7]:
relevant_data = admissions_per_subject[admissions_per_subject['num_admissions'] >= 2]
relevant_data

,hadm_id,admittime,diagnosis,icd9_code,icd9_code_converted,icd9_code_converted_3digits,num_admissions
subject_id,,,,,,,
10059,"[142582, 122098]","[2150-08-07 21:40:00, 2150-08-22 17:33:00]","[VARICEAL BLEED, LOWER GI BLEED]","[[5715, 5849, 07070, 2800, 2875, 7895, 45620, ...","[[571.5, 584.9, 070.70, 280.0, 287.5, 789.5, 4...","[[571, 584, 070, 280, 287, 789, 456, 572, 401]...",2
10088,"[169938, 168233, 149044]","[2107-01-04 11:59:00, 2107-01-29 04:00:00, 210...","[SEPSIS;PNEUMONIA;TELEMETRY, CONGESTIVE HEART ...","[[0389, 99592, 5990, 78552, 51881, 4280, 2765,...","[[038.9, 995.92, 599.0, 785.52, 518.81, 428.0,...","[[038, 995, 599, 785, 518, 428, 276, 578, 041,...",3
10094,"[168074, 122928]","[2180-02-29 18:54:00, 2180-03-15 22:35:00]","[HYPOTENSION;TELEMETRY, SEPSIS;TELEMETRY]","[[2273, 486, 5849, 2532, 3970, 4280, 2760, 707...","[[227.3, 486, 584.9, 253.2, 397.0, 428.0, 276....","[[227, 486, 584, 253, 397, 428, 276, 707, 255,...",2
10117,"[187023, 105150]","[2138-06-05 17:23:00, 2138-11-09 18:08:00]","[FEVER, FEVER]","[[5290, 7907, 2762, 20300, 2875], [486, 99662,...","[[529.0, 790.7, 276.2, 203.00, 287.5], [486, 9...","[[529, 790, 276, 203, 287], [486, 996, 790, 51...",2
10119,"[157466, 165436]","[2117-08-05 18:27:00, 2117-08-21 06:58:00]","[ACUTE CHOLECYSTITIS, GASTROINTESTINAL BLEED]","[[5750, 5754, 42731, 7907, 5770, 5849, 5185, 3...","[[575.0, 575.4, 427.31, 790.7, 577.0, 584.9, 5...","[[575, 575, 427, 790, 577, 584, 518, 349, 276,...",2
10124,"[182664, 170883]","[2192-03-26 15:30:00, 2192-04-16 20:57:00]","[LEFT HIP FRACTURE, CONGESTIVE HEART FAILURE]","[[82021, 42832, 4280, 5990, 2851, 42731, 5859,...","[[820.21, 428.32, 428.0, 599.0, 285.1, 427.31,...","[[820, 428, 428, 599, 285, 427, 585, 998, 584,...",2
40124,"[126179, 146893]","[2130-02-04 02:26:00, 2130-08-12 05:49:00]","[SHORTNESS OF BREATH, PNEUMONIA]","[[486, 5849, 1628, 27651, 2859, 496, 4019], [4...","[[486, 584.9, 162.8, 276.51, 285.9, 496, 401.9...","[[486, 584, 162, 276, 285, 496, 401], [486, 51...",2
40310,"[186361, 157609]","[2144-07-11 15:02:00, 2144-12-24 16:16:00]","[FACIAL NUMBNESS, AROMEGLEY;BURKITTS LYMPHOMA]","[[20020, 51881, 27788, 5845, 48242, 70724, 284...","[[200.20, 518.81, 277.88, 584.5, 482.42, 707.2...","[[200, 518, 277, 584, 482, 707, 284, 483, 453,...",2
41795,"[138132, 118192]","[2145-07-07 01:19:00, 2145-09-06 08:52:00]","[ASTHMA;CHRONIC OBST PULM DISEASE, ASTHMA;CHRO...","[[51881, 49121, 42833, 486, 2761, 29562, 2841,...","[[518.81, 491.21, 428.33, 486, 276.1, 295.62, ...","[[518, 491, 428, 486, 276, 295, 284, 276, 428,...",2


In [8]:
all_symptoms = list(set([item for sublist in relevant_data['icd9_code_converted_3digits'].agg(lambda x: [item for sublist in x for item in sublist]).tolist() for item in sublist]))
vocab = {}
index = 0
for symptom in all_symptoms:
    vocab[symptom] = index
    index = index+1

len(vocab)

150

In [9]:
max_sequence_length = relevant_data['icd9_code_converted_3digits'].apply(len).max()
max_symptoms_per_sequence = relevant_data['icd9_code_converted_3digits'].apply(lambda x: sum([len(y) for y in x])).max()
train_sequences, test_sequences = train_test_split(
    relevant_data['icd9_code_converted_3digits'], 
    test_size=0.1, 
    random_state=12345)
train_sequences.tolist()[5]

[['151', '518', '401', '272'],
 ['511', '423', '401', '285', 'V10'],
 ['423', '511', '423', '285', 'V10', '401']]

In [10]:
def split_sequence(sequence):
    splitted = []
    for split_index in range(1, len(sequence)):
        splitted.append({
            'x': sequence[0:split_index],
            'y': sequence[split_index], 
        })

    return splitted

def split_sequences(sequences):
    splitted_sequences = []
    for sequence in sequences:
        splitted_sequences.extend(split_sequence(sequence))

    return splitted_sequences

def transform_symptoms(symptoms, vocab):
    symptom_vec = np.zeros(len(vocab))
    for symptom in symptoms:
        symptom_vec[vocab[symptom]] = 1
    return tf.convert_to_tensor(symptom_vec)

def translate_and_pad_x_flat(splitted, vocab, max_sequence_length):
    splitted['x_vecs'] = []
    for i in range(max_sequence_length - len(splitted['x'])):
        splitted['x_vecs'].append(transform_symptoms([], vocab))
    for x in splitted['x']:
        splitted['x_vecs'].append(transform_symptoms(x, vocab))
    splitted['x_vecs_stacked'] = tf.stack(splitted['x_vecs'])

def translate_and_pad_x_wide(splitted, vocab, max_symptoms_per_sequence):
    splitted['x_vecs'] = []
    all_symptoms = [symptom for x in splitted['x'] for symptom in x]
    for i in range(max_symptoms_per_sequence - len(all_symptoms)):
        splitted['x_vecs'].append(transform_symptoms([], vocab))
    for symptom in all_symptoms:
        splitted['x_vecs'].append(transform_symptoms([symptom], vocab))
    splitted['x_vecs_stacked'] = tf.stack(splitted['x_vecs'])

def translate_and_pad(splitted, vocab, max_sequence_length, max_symptoms_per_sequence, flat):
    splitted['y_vec'] = transform_symptoms(splitted['y'], vocab)
    if flat:
        translate_and_pad_x_flat(splitted, vocab, max_sequence_length)
    else:
        translate_and_pad_x_wide(splitted, vocab, max_symptoms_per_sequence)
    

def transform_sequences(sequences, vocab, max_sequence_length, max_symptoms_per_sequence, flat=True):
    splitted_sequences = split_sequences(sequences)
    for splitted in splitted_sequences:
        translate_and_pad(splitted, vocab, max_sequence_length, max_symptoms_per_sequence, flat)

    return splitted_sequences


transformed_5 = transform_sequences(train_sequences.tolist()[5:6], vocab, max_sequence_length, max_symptoms_per_sequence, flat=False)
tf.stack([[transformed['y_vec']] for transformed in transformed_5])
tf.stack([transformed['x_vecs_stacked'] for transformed in transformed_5])

<tf.Tensor: shape=(2, 266, 150), dtype=float64, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])>

In [11]:
train_transformed_flat = transform_sequences(train_sequences.tolist(), vocab, max_sequence_length, max_symptoms_per_sequence, flat=True)
test_transformed_flat = transform_sequences(test_sequences.tolist(), vocab, max_sequence_length, max_symptoms_per_sequence, flat=True)

train_x_flat = tf.stack([transformed['x_vecs_stacked'] for transformed in train_transformed_flat])
train_y_flat = tf.stack([[transformed['y_vec']] for transformed in train_transformed_flat])
test_x_flat = tf.stack([transformed['x_vecs_stacked'] for transformed in test_transformed_flat])
test_y_flat = tf.stack([[transformed['y_vec']] for transformed in test_transformed_flat])

print(train_x_flat.shape)
print(train_y_flat.shape)

(27, 15, 150)
(27, 1, 150)


In [12]:
train_transformed_wide = transform_sequences(train_sequences.tolist(), vocab, max_sequence_length, max_symptoms_per_sequence, flat=False)
test_transformed_wide = transform_sequences(test_sequences.tolist(), vocab, max_sequence_length, max_symptoms_per_sequence, flat=False)

train_x_wide = tf.stack([transformed['x_vecs_stacked'] for transformed in train_transformed_wide])
train_y_wide = tf.stack([[transformed['y_vec']] for transformed in train_transformed_wide])
test_x_wide = tf.stack([transformed['x_vecs_stacked'] for transformed in test_transformed_wide])
test_y_wide = tf.stack([[transformed['y_vec']] for transformed in test_transformed_wide])

print(train_x_wide.shape)
print(train_y_wide.shape)

(27, 266, 150)
(27, 1, 150)


In [13]:
input_layer_flat = tf.keras.layers.Input(shape=(max_sequence_length,len(vocab)))
emb_layer_flat = tf.keras.layers.Dense(64)
lstm_model_flat = tf.keras.models.Sequential([
    input_layer_flat,
    emb_layer_flat,
    tf.keras.layers.LSTM(32, return_sequences=False),
    #tf.keras.layers.Flatten(),
    #tf.keras.layers.Conv1D(filters=32,
    #                       kernel_size=(3,),
    #                       activation='relu'),
    tf.keras.layers.Dense(len(vocab), activation='relu'),
])
lstm_model_flat.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.optimizers.Adam())
lstm_model_flat.fit(x=train_x_flat, y=train_y_flat, epochs=100)

emb_model_flat = tf.keras.models.Sequential([
    input_layer_flat,
    emb_layer_flat,
])
print(emb_model_flat.predict(train_x_flat).shape)

lstm_model_flat.evaluate(test_x_flat, test_y_flat)

Epoch 1/100
1/1 [==============================] - 0s 997us/step - loss: 0.9145
Epoch 2/100
1/1 [==============================] - 0s 1000us/step - loss: 0.8666
Epoch 3/100
1/1 [==============================] - 0s 1ms/step - loss: 0.8317
Epoch 4/100
1/1 [==============================] - 0s 999us/step - loss: 0.8050
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 0.7788
Epoch 6/100
1/1 [==============================] - 0s 1ms/step - loss: 0.7617
Epoch 7/100
1/1 [==============================] - 0s 998us/step - loss: 0.7460
Epoch 8/100
1/1 [==============================] - 0s 0s/step - loss: 0.7268
Epoch 9/100
1/1 [==============================] - 0s 998us/step - loss: 0.7097
Epoch 10/100
1/1 [==============================] - 0s 999us/step - loss: 0.6940
Epoch 11/100
1/1 [==============================] - 0s 998us/step - loss: 0.6785
Epoch 12/100
1/1 [==============================] - 0s 998us/step - loss: 0.6641
Epoch 13/100
1/1 [===========================

0.9456964135169983

In [14]:
input_layer_wide = tf.keras.layers.Input(shape=(max_symptoms_per_sequence,len(vocab)))
emb_layer_wide = tf.keras.layers.Dense(64)
lstm_model_wide = tf.keras.models.Sequential([
    input_layer_wide,
    emb_layer_wide,
    tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.Dense(len(vocab), activation='relu'),
])
lstm_model_wide.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.optimizers.Adam())
lstm_model_wide.fit(x=train_x_wide, y=train_y_wide, epochs=100)

emb_model_wide = tf.keras.models.Sequential([
    input_layer_wide,
    emb_layer_wide,
])
print(emb_model_wide.predict(train_x_wide).shape)

lstm_model_wide.predict(test_x_wide)

Epoch 1/100
1/1 [==============================] - 0s 1ms/step - loss: 0.9821
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 0.9128
Epoch 3/100
1/1 [==============================] - 0s 996us/step - loss: 0.8461
Epoch 4/100
1/1 [==============================] - 0s 996us/step - loss: 0.7847
Epoch 5/100
1/1 [==============================] - 0s 0s/step - loss: 0.7356
Epoch 6/100
1/1 [==============================] - 0s 0s/step - loss: 0.6977
Epoch 7/100
1/1 [==============================] - 0s 996us/step - loss: 0.6595
Epoch 8/100
1/1 [==============================] - 0s 995us/step - loss: 0.6260
Epoch 9/100
1/1 [==============================] - 0s 990us/step - loss: 0.5980
Epoch 10/100
1/1 [==============================] - 0s 996us/step - loss: 0.5773
Epoch 11/100
1/1 [==============================] - 0s 997us/step - loss: 0.5548
Epoch 12/100
1/1 [==============================] - 0s 3ms/step - loss: 0.5385
Epoch 13/100
1/1 [==============================]

array([[0.        , 0.35038397, 0.        , 0.07885708, 0.        ,
        0.        , 0.        , 0.01622912, 0.2663327 , 0.        ,
        0.0955881 , 0.        , 0.04688806, 0.2964153 , 0.03177335,
        0.10052165, 0.11688656, 0.06507315, 0.        , 0.        ,
        0.03614247, 0.0739017 , 0.32606223, 0.07919644, 0.        ,
        0.04697911, 0.02439923, 0.12501813, 0.08061726, 0.22944619,
        0.08373684, 0.34498924, 0.08079305, 0.01633723, 0.03927398,
        0.31870085, 0.        , 0.        , 0.07873262, 0.1718489 ,
        0.07555813, 0.07685293, 0.        , 0.04561304, 0.        ,
        0.04555793, 0.        , 0.04544513, 0.        , 0.01481071,
        0.03420307, 0.03207786, 0.09392075, 0.36987105, 0.        ,
        0.01485855, 0.09945916, 0.        , 0.15017149, 0.        ,
        0.03478513, 0.01393906, 0.        , 0.04392799, 0.        ,
        0.03402762, 0.11996071, 0.07216411, 0.02995195, 0.07507603,
        0.3634999 , 0.        , 0.04285083, 0.  

In [15]:
prediction = lstm_model_wide.predict(test_x_wide)[0]
np.argwhere(prediction > 0.5)

array([[130]], dtype=int64)

In [16]:
np.argwhere(test_y_wide[1] == 1)

array([[  0,   4],
       [  0,  11],
       [  0,  12],
       [  0,  13],
       [  0,  24],
       [  0,  25],
       [  0,  29],
       [  0,  43],
       [  0,  56],
       [  0,  70],
       [  0,  78],
       [  0,  86],
       [  0,  87],
       [  0,  90],
       [  0,  91],
       [  0,  96],
       [  0, 117],
       [  0, 118],
       [  0, 119],
       [  0, 128],
       [  0, 132],
       [  0, 135]], dtype=int64)

In [17]:
[(x, vocab[x]) for x in vocab.keys() if vocab[x] in [7, 26]]

[('287', 7), ('790', 26)]